In [0]:
import math
import os
import pandas as pd
import re
import urllib
import bs4
import math
import time

In [0]:
class DataScraper:

    def __init__(self, product_type):
    
        self.product_type = product_type
        

    def GetPageCount(self):

        url = 'https://in.bodybuilding.com/store/'+self.product_type+'.html?filter=product_form:Powder&page=1&includePrevPageItems=false'
        req = urllib.request.Request(url , headers = {'User-Agent' : 'Chrome/41.0.2228.0'})
        src = urllib.request.urlopen(req)
        sp = bs4.BeautifulSoup(src)
        
        tot_items = str(sp.find('div' , {'class' : "results"}).get_text(strip = True)).split(' ')[0]
        print(f"The total item of {self.product_type} catgeory is {tot_items}")
        
        return math.ceil((int(tot_items)/20))

    def get_links(self):
        
        total_pages = self.GetPageCount()

        links = []
        for pg in range(total_pages):

            url = 'https://in.bodybuilding.com/store/' + self.product_type +'.html?filter=product_form:Powder&page='+ str(pg) + '&includePrevPageItems=false'
            req = urllib.request.Request(url , headers = {'User-Agent' : 'Chrome/41.0.2228.0'})
            src = urllib.request.urlopen(req)
            soup = bs4.BeautifulSoup(src , 'lxml')

            for tag in soup.find_all('a' , {'class' : "product__mobile-click-target"}):
                links.append(tag['href'])
        print(len(links))

        return links

    def CollectData(self):
        
        data ={}
        links = self.get_links()
        df = pd.DataFrame(columns = ['Product' , 'Brand' ,'Price','Rating' ,'Review_Count' ,'Flavor_Rating', 'Flavor','Size', 'Servings',  
                                     'Scoop_Size' , 'Available_Sizes' , 'Available_Flavors'])
        for link in links:
            req = urllib.request.Request(link , headers = {'User-Agent' : 'Chrome/41.0.2228.0'})
            src = urllib.request.urlopen(req)
            soup = bs4.BeautifulSoup(src , 'lxml')
            #time.sleep(5)

            try:
                product = soup.find('h1' , {'class' : "Product__name"}).get_text(strip = True)
            except:
                product = None
            
            try:
                brand = soup.find('a' , {'class' : "Product__brand"}).get_text(strip = True)
            except:
                brand = None

            try:
                price = float(soup.find('span' , {'class' : "sku-chooser__sale-price"}).get_text(strip = True).strip('$'))
            except:
                price = None

            try:
                rating = float(soup.find('span' ,{'class' : "RatingBadge-sm"}).get_text(strip = True))
            except:
                rating = None
            #time.sleep(2)
            try:
                review_count = soup.find('a' ,{'class' : "RatingBadge__read-more"})
                review_count = int(review_count.text.strip().split(' ')[0].strip('(').replace(',',''))
                #print(review_x)
            except:
                #print("Entering Except")
                review_count = None
            
            
            try:
                flavor_rating = float( soup.find('div' , {'class' : 'sku-chooser__flavor-rating'}).find('strong').get_text(strip = True))
            except:
                flavor_rating = None

            try:
                vals = soup.find_all('div' , {'class' : "sku-chooser__label-super"})
                elements = []
                for ele in vals:
                    if len(ele.text.strip()) != 0:
                        elements.append(ele.text.strip())
                
                ava_size = int(elements[0].split(' ')[0])
                #print(ava_size)
                ava_flavors = int(elements[1].split(' ')[0])
                #print(ava_flavors)
            except:
                ava_size , ava_flavors = None, None

            try:
                vals = soup.find_all('div' , {'class' : "sku-chooser__label-value"})
                elements = []
                for ele in vals:
                    if ele.text.strip().find('vm') == -1:
                        elements.append(ele.text.strip())
                        
                size = elements[0]
                flavor = elements[1]
            except:
                size , flavor = None , None

            try:
                vals = soup.find_all('td' , {'class' : "seq_span label_serving"})
                for ele in vals:
                    if  ele.text.find('Scoop') != -1 or ele.text.find('scoop') != -1:
                        scoop_size = ele.text.strip()
                        #print(scoop_size)
                    elif ele.text.find('Container') != -1:
                        servings = ele.text.strip()
                        #print(servings)
            except:
                scoop_size , servings = None , None

            df = df.append({'Product' : product,
                            'Brand' : brand,
                            'Price' : price,
                            'Rating' : rating , 
                            'Review_Count' : review_count ,
                            'Flavor_Rating' : flavor_rating , 
                            'Flavor' : flavor,
                            'Size' : size , 
                            'Servings' : servings , 
                            'Scoop_Size' : scoop_size , 
                            'Available_Sizes' : ava_size,
                            'Available_Flavors' : ava_flavors} , ignore_index = True)
        return df

In [14]:
x = DataScraper('improve-workouts')
df = x.CollectData()

The total item of improve-workouts catgeory is 269
269


In [15]:
df.head()

,Product,Brand,Price,Rating,Review_Count,Flavor_Rating,Flavor,Size,Servings,Scoop_Size,Available_Sizes,Available_Flavors
0,Pre JYM Pre Workout Powder,JYM Supplement Science,38.99,9.1,4109,9,Grape Candy,30 Servings,Servings Per Container: About 30,Serving Size: 1 Scoop (25g),2,11
1,Post JYM Recovery and Active BCAAs Powder,JYM Supplement Science,32.99,9.5,1410,9.5,Watermelon,30 Servings,Servings Per Container: About30,Serving Size: 1 Scoop (21g),1,7
2,Essential AmiN.O. Energy,Optimum Nutrition,16.49,9.1,2499,9.4,Juicy Strawberry Burst,30 Servings,Servings Per Container30,Serving Size 2 Scoops (9 g),4,14
3,BCAA Energy Amino Acids,Evlution Nutrition,21.99,9.4,2861,9.7,Pink Starblast,30 Servings,Servings Per Container30,Serving Size 2 Scoops (9g),2,11
4,ENGN Shred Pre Workout,Evlution Nutrition,36.99,9.3,1537,9.4,Pink Lemonade,30 Servings,Servings Per Container30,Serving Size 1 Scoop (7.1g),1,5


In [16]:
df.shape

(269, 12)

In [0]:
df.to_csv('Improve_workouts.csv', index = False)